In [102]:
from twython import Twython
from time import sleep
import json
import pandas as pd
import io
from tqdm import tqdm
import re # regular expressions
from tqdm import tnrange, tqdm_notebook, tqdm

In [103]:
currency = "bitcoin"
symbol = "BTC"

# Configuration
folder = f"data/crypto/{symbol}" # Relative path to historical data
separating_character = '~' # character seperating dates from and to in filename
anvs = ['CRYPTO', 'LINE_COUNT', 'MOST_RECENT_FILE', 'MOST_RECENT_ID'] # Stored in var.csv
max_rows = 20000 # Each file storing data has a maximum amount of rows

tweets_raw_file = f'../data/twitter/{symbol}/{currency}_tweets_raw.csv'
tweets_clean_file = f'../data/twitter/{symbol}/{currency}_tweets_clean.csv'
query = f'#{currency} OR #{symbol}' ####TODO PUT BACK  OR {CURRENCY} OR ${CURRENCY} OR ${CURRENCY_SYMBOL}

In [104]:
app_key = '2IGXQ4kdoysrGMeaUZkSvGCvM'
app_secret = 'ZS6drXCpvUlQJhdQ7g3XbgCaOLI5IdPeA6fKwtEGkTIrx5YHaR'

OAUTH_TOKEN=auth['oauth_token']
OAUTH_TOKEN_SECRET=auth['oauth_token_secret']

twitter = Twython(app_key, app_secret, oauth_version=2)
ACCESS_TOKEN = twitter.obtain_access_token()
twitter = Twython(app_key, access_token=ACCESS_TOKEN)
twitter.get_application_rate_limit_status()['resources']['search']

{'/search/tweets': {'limit': 450, 'remaining': 85, 'reset': 1641867005}}

In [105]:
queries_number = 1
data = {"statuses": []}
next_id = "1"
print(range(queries_number))
print(queries_number)
with open(tweets_raw_file,"a+", encoding='utf-8') as f:
    if not next_id:
        f.write("ID,Text,UserName,UserFollowerCount,RetweetCount,Likes,CreatedAt\n")
    while(True):
        twitter = Twython(app_key, access_token=ACCESS_TOKEN)
        last_size = 0
        for i in range(queries_number):
            if not next_id:
                data = twitter.search(q=query, lang='en', result_type='recent', count="100") # Use since_id for tweets after id
            else:
                data["statuses"].extend(twitter.search(q=query, lang='en', result_type='mixed', count="100", max_id=next_id)["statuses"])
            if len(data["statuses"]) > 1:
                next_id = data["statuses"][len(data["statuses"]) - 1]['id']
            if last_size + 1 == len(data["statuses"]):
                break
            else:
                last_size = len(data["statuses"])

        print('Retrieved {0}, waiting for 15 minutes until next queries'.format(len(data["statuses"])))
        d = pd.DataFrame([[s["id"], s["text"].replace('\n','').replace('\r',''), s["user"]["name"], s["user"]["followers_count"], s["retweet_count"], s["favorite_count"], s["created_at"]] for s in data["statuses"]], columns=('ID', 'Text', 'UserName', "UserFollowerCount", 'RetweetCount', 'Likes', "CreatedAt"))
        d.to_csv(f, mode='a', encoding='utf-8',index=False,header=False)
        if last_size + 1 < len(data["statuses"]):
            print('No more new tweets, stopping...')
            break
        data["statuses"] = []
        
        sleep(910)

range(0, 1)
1
Retrieved 93, waiting for 15 minutes until next queries


KeyboardInterrupt: 

In [106]:
d = pd.read_csv(tweets_raw_file)
for i,s in enumerate(tqdm(d['Text'])):
    text = d.loc[i, 'Text']
    text = text.replace("#", "")
    text = re.sub('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', '', text, flags=re.MULTILINE)
    text = re.sub('@\\w+ *', '', text, flags=re.MULTILINE)
    d.loc[i, 'Text'] = text
f = open(tweets_clean_file, 'a+', encoding='utf-8')
d.to_csv(f, header=True, encoding='utf-8',index=False)

100%|██████████| 93/93 [00:00<00:00, 2885.95it/s]


In [107]:
df_clean = pd.read_csv(tweets_clean_file)
df_clean.head(5)

,ID,Text,UserName,UserFollowerCount,RetweetCount,Likes,CreatedAt
0,ID,Text,UserName,UserFollowerCount,RetweetCount,Likes,CreatedAt
1,1480507769610510343,Legendary investor Bill Miller is still buying...,Bitcoin Magazine,1921879,648,3898,Mon Jan 10 11:52:13 +0000 2022
2,1480190307224465408,Stay hungry. Bitcoin,Michael Saylor⚡️,2084920,2017,21352,Sun Jan 09 14:50:44 +0000 2022
3,1480259013455400961,1/ BTC 🤝 ICP direct integration (🚫🌉)cryptograp...,DFINITY Foundation,463883,448,1173,Sun Jan 09 19:23:45 +0000 2022
4,1480722463679860736,RT : Gold has gone from 0 to $1850 in 5000 yea...,Mr.BUIDL🦇🔊,503,607,0,Tue Jan 11 02:05:20 +0000 2022


In [108]:
df_clean.min(axis=0)

ID                                                 1480190307224465408
Text                 "Give me your tired, your poor,Your huddled ma...
UserName                                                    @Yogasedaw
UserFollowerCount                                                    0
RetweetCount                                                         0
Likes                                                                0
CreatedAt                                                    CreatedAt
dtype: object

In [109]:
f.close()